In [1]:
import numpy as np

1. Free space path loss: https://en.wikipedia.org/wiki/Free-space_path_loss
2. Shadow Fading loss: avg -> 1
3. Anttena Gain: 30 dBi

In [3]:
# 1. Free space path loss: https://en.wikipedia.org/wiki/Free-space_path_loss
d = 500 # [km]
f = 14 # [14 GHz]
FSPL = 20 * np.log10(d) + 20 * np.log10(f) + 92.45 # [dB]

# 2. Shadow Fading loss: avg -> 1
shadow = 1

# 3. Anttena Gain: 30 dBi: 논문 참조 joint ~, IEEE Access
anttena_gain = 30 # [dBi]

# 4. Noise temperatur of LEO -> 550 K convert to Noise: https://www.everythingrf.com/rf-calculators/noise-temperature-noise-figure-calculator
noise_temperature = 550
noise_power = 10 * np.log10(noise_temperature / 290 + 1) # [dB]

In [2]:
def cal_signal_power(SAT, GS, coverage_indicator, freq, speed):
    """
    return uplink signal power

    shadow faiding loss -> avg 1
    """
    anttena_gain = 30 # [dBi]
    GS_Tx_power = 23e-3 # 23 dBm

    GS_signal_power = np.zeros(len(GS))
    for i in range(len(GS)):
            for j in range(len(SAT)):
                if coverage_indicator[i][j]:
                    dist = np.linalg.norm(GS[i,0:2] - SAT[j,0:2]) # 2-dim 
                    f = freq * np.abs(speed) * (dist / (GS[i,0]-SAT[i,0])) / (3e5) # Doppler shift !단위 주의!
                    FSPL = 20 * np.log10(dist) + 20 * np.log10(f) + 92.45 # [dB], free space path loss
                    GS_signal_power[i] = GS_Tx_power * (FSPL + anttena_gain)

    return GS_signal_power

In [3]:
def cal_SINR(GS_index, SAT_serviced_indicator, signal_power, noise_temperature = 550):
    """
    Input parameter:
        noise_temperature: 550 [K]
    return uplink SINR
    """
    SINR = 0 # [dB]
    
    noise_power = 10 * np.log10(noise_temperature / 290 + 1) # [dB]

    if len(SAT_serviced_indicator > 1):
        interference = np.sum(signal_power[SAT_serviced_indicator]) - signal_power[GS_index]
        SINR = signal_power[GS_index] / (interference + noise_power)
    else:
        SINR = signal_power[GS_index] / noise_power
    
    return SINR

